In [90]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [91]:
class modelNet:
    @staticmethod
    def LeNet(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(8, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(16, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(100))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model
    
    @staticmethod
    def VGG16(classes):
        # initializing the model
        model = Sequential()
        
        
        model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Flatten())
        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=4096,activation="relu"))
        
        model.add(Dense(units=classes, activation="softmax"))
    

In [92]:
EPOCHS = 15
INIT_LR = 1e-3
BS = 64
CLASSES = 3

In [93]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [94]:
imagePaths = sorted(list(paths.list_images("augData/trialDataset")))
random.seed(42)
random.shuffle(imagePaths)

In [95]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    

In [96]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{'double_person': 947, 'single_person': 963, 'triple_person_3': 612}

In [97]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [98]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [99]:
print("[INFO] compiling model...")
model = modelNet.LeNet(width=28, height=28, depth=3, classes=CLASSES)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [100]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 8)         608       
                                                                 
 activation_16 (Activation)  (None, 28, 28, 8)         0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 8)        0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 16)        3216      
                                                                 
 activation_17 (Activation)  (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                  

In [101]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
    validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/15
29/29 [==============================] - 5s 137ms/step - loss: 0.9342 - accuracy: 0.5210 - val_loss: 0.8038 - val_accuracy: 0.6006
Epoch 2/15
29/29 [==============================] - 2s 77ms/step - loss: 0.7206 - accuracy: 0.6612 - val_loss: 0.7406 - val_accuracy: 0.6070
Epoch 3/15
29/29 [==============================] - 2s 86ms/step - loss: 0.6286 - accuracy: 0.7132 - val_loss: 0.6568 - val_accuracy: 0.6640
Epoch 4/15
29/29 [==============================] - 3s 114ms/step - loss: 0.5889 - accuracy: 0.7334 - val_loss: 0.6038 - val_accuracy: 0.6941
Epoch 5/15
29/29 [==============================] - 3s 114ms/step - loss: 0.5660 - accuracy: 0.7258 - val_loss: 0.5643 - val_accuracy: 0.7195
Epoch 6/15
29/29 [==============================] - 3s 111ms/step - loss: 0.5423 - accuracy: 0.7603 - val_loss: 0.5852 - val_accuracy: 0.7163
Epoch 7/15
29/29 [==============================] - 3s 115ms/step - loss: 0.5129 - accuracy: 0.7690 - val_loss: 0.5858 - va

In [102]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_multi_class_4.png")

In [103]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(631, 3)


In [104]:
predictions

array([[6.9754297e-01, 2.9562002e-01, 6.8370001e-03],
       [4.4647187e-01, 5.0756234e-01, 4.5965761e-02],
       [9.4758216e-03, 8.4498766e-05, 9.9043965e-01],
       ...,
       [2.5694194e-01, 7.3301589e-01, 1.0042241e-02],
       [1.7739239e-01, 8.1582993e-01, 6.7776656e-03],
       [7.3558606e-02, 9.2508733e-01, 1.3539516e-03]], dtype=float32)

In [105]:
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=lb.classes_))

                 precision    recall  f1-score   support

  double_person       0.75      0.69      0.72       251
  single_person       0.74      0.87      0.80       239
triple_person_3       0.94      0.79      0.86       141

       accuracy                           0.78       631
      macro avg       0.81      0.78      0.79       631
   weighted avg       0.79      0.78      0.78       631



### Trying out test set with 3 classification

In [106]:
test_imagepaths = sorted(list(paths.list_images("testSet")))
random.seed(42)
random.shuffle(test_imagepaths)

In [107]:
test_labels = []
test_data = []
for imagePath in test_imagepaths:    
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    test_data.append(image)

test_labels = np.array(test_labels)
test_data = np.array(test_data, dtype="float") / 255.0


In [108]:
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

In [109]:
test_predictions = model.predict(x=test_data, batch_size=32)

print(test_predictions.shape)

(221, 3)


In [110]:
test_predictions.argmax(axis=1)

array([0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 1, 0, 1, 0, 0, 2, 2, 2, 2, 0,
       0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 2, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 2, 2, 0, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 1, 2, 1, 0, 1,
       1, 0, 2, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 0, 0, 1, 1, 1, 2, 0, 1, 2,
       0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 1, 2, 0, 2, 2, 0, 1, 2, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 2, 2, 2, 0, 1, 1, 2, 1, 1, 2, 0, 0, 2, 0, 1, 2,
       0, 2, 0, 1, 0, 0, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1,
       0])

In [111]:
print(classification_report(test_labels.argmax(axis=1),test_predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.26      0.40      0.32        78
single_person       0.75      0.55      0.63        80
triple_person       0.20      0.14      0.17        63

     accuracy                           0.38       221
    macro avg       0.40      0.36      0.37       221
 weighted avg       0.42      0.38      0.39       221

